# 📘 Introduction: Loop Unrolling and Register Optimization

Modern GPU performance is often limited not only by memory bandwidth, but also by **instruction throughput and instruction scheduling efficiency**.  
This work focuses on two fundamental low-level optimization techniques in CUDA: **loop unrolling** and **register optimization**, both of which aim to reduce instruction overhead and increase computational efficiency.

---

## 🔁 Loop Unrolling

Loop unrolling is an optimization technique that **expands the loop body multiple times**, reducing the number of loop-control instructions such as:
- loop counters
- branch instructions
- index updates

On GPUs, loop unrolling can:
- Increase **instruction-level parallelism (ILP)**
- Reduce branch overhead
- Enable better compiler scheduling

In compute-heavy kernels (e.g., dot products, reductions, matrix multiplication), unrolling inner loops often leads to **higher throughput and lower latency**.

---

## 🧮 Register Optimization

Registers are the **fastest storage** available to GPU threads.  
Register optimization focuses on:
- Maximizing data reuse in registers
- Minimizing unnecessary register spills to local memory
- Balancing register usage to maintain high occupancy

Effective register usage:
- Reduces memory accesses
- Improves arithmetic efficiency
- Avoids performance loss due to register spilling

---

## 🎯 Purpose of This Work

The goal of this work is to:
- Apply loop unrolling to critical compute loops
- Analyze how unrolling affects register usage and performance
- Optimize register allocation to improve throughput without harming occupancy

Through these optimizations, you will learn how **compute-bound kernels can be significantly accelerated**, and how low-level code structure influences GPU performance.

---

## 🚀 Relevance to ML Systems

Loop unrolling and register optimization are widely used in:
- GEMM and convolution kernels
- Reduction and normalization layers
- Attention and FlashAttention kernels
- Compiler-generated kernels (e.g., Triton, TVM)

Understanding these techniques helps bridge the gap between **naive CUDA kernels** and **production-grade ML kernels**.

---

## 🧠 Key Takeaway

> **Loop unrolling reduces instruction overhead, while register optimization ensures that frequently used data stays in the fastest possible storage. Together, they are essential for achieving high-performance GPU kernels.**


In [1]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
Mon Jan 19 12:17:45 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:00:05.0 Off |                    0 |
| N/A   33C    P0       

In [24]:
%%writefile unroll_register_compare.cu
#include <cstdio>
#include <cstdlib>
#include <cmath>
#include <cuda_runtime.h>

#define CUDA_CHECK(call) do {                                   \
  cudaError_t err = (call);                                     \
  if (err != cudaSuccess) {                                     \
    fprintf(stderr, "CUDA error %s:%d: %s\n",                   \
            __FILE__, __LINE__, cudaGetErrorString(err));       \
    std::exit(EXIT_FAILURE);                                    \
  }                                                             \
} while(0)

// -----------------------------
// Tune these
// -----------------------------
#ifndef UNROLL
#define UNROLL 4
#endif

#ifndef MAX_K
#define MAX_K 2048   // must be >= K; constant memory size fixed at compile time
#endif

// B in constant memory (read-only, broadcast friendly)
__constant__ float Bc[MAX_K];

// ------------------------------------------------------------
// Baseline kernel
// out[i] = sum_k A[i*K + k] * B[k]
// ------------------------------------------------------------
__global__ void dotBaseline(const float* __restrict__ A,
                            const float* __restrict__ B,
                            float* __restrict__ out,
                            int M, int K) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i >= M) return;

    const float* Ai = A + (size_t)i * K;
    float acc = 0.0f;

    for (int k = 0; k < K; ++k) {
        acc += Ai[k] * B[k];
    }
    out[i] = acc;
}

// ------------------------------------------------------------
// Unroll + Register kernel (B in global memory)
// ------------------------------------------------------------
__global__ void dotUnrollRegisterGlobalB(const float* __restrict__ A,
                                        const float* __restrict__ B,
                                        float* __restrict__ out,
                                        int M, int K) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i >= M) return;

    const float* Ai = A + (size_t)i * K;
    float acc = 0.0f;

    int k = 0;
    for (; k + UNROLL - 1 < K; k += UNROLL) {
        #pragma unroll
        for (int u = 0; u < UNROLL; ++u) {
            float a = Ai[k + u];   // registers
            float b = B[k + u];    // registers
            acc += a * b;
        }
    }
    for (; k < K; ++k) {
        acc += Ai[k] * B[k];
    }
    out[i] = acc;
}

// ------------------------------------------------------------
// Unroll + Register + Constant memory kernel (B in Bc[])
// ------------------------------------------------------------
__global__ void dotUnrollRegisterConstB(const float* __restrict__ A,
                                       float* __restrict__ out,
                                       int M, int K) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    if (i >= M) return;

    const float* Ai = A + (size_t)i * K;
    float acc = 0.0f;

    int k = 0;
    for (; k + UNROLL - 1 < K; k += UNROLL) {
        #pragma unroll
        for (int u = 0; u < UNROLL; ++u) {
            float a = Ai[k + u];
            float b = Bc[k + u];   // const cache broadcast friendly
            acc += a * b;
        }
    }
    for (; k < K; ++k) {
        acc += Ai[k] * Bc[k];
    }
    out[i] = acc;
}

// ------------------------------------------------------------
// CPU reference
// ------------------------------------------------------------
static void dotCPU(const float* A, const float* B, float* out, int M, int K) {
    for (int i = 0; i < M; ++i) {
        double acc = 0.0;
        const float* Ai = A + (size_t)i * K;
        for (int k = 0; k < K; ++k) acc += (double)Ai[k] * (double)B[k];
        out[i] = (float)acc;
    }
}

// ------------------------------------------------------------
// Correctness checker
// ------------------------------------------------------------
static bool checkClose(const float* gpu, const float* ref, int n, float tol) {
    for (int i = 0; i < n; ++i) {
        float diff = fabsf(gpu[i] - ref[i]);
        float denom = fmaxf(1.0f, fabsf(ref[i]));
        if (diff / denom > tol) {
            printf("Mismatch at %d: ref=%.6f gpu=%.6f (rel=%.6e)\n",
                   i, ref[i], gpu[i], diff / denom);
            return false;
        }
    }
    return true;
}

// ------------------------------------------------------------
// Timing helper
// ------------------------------------------------------------
template <typename KernelFunc>
static float timeKernelMs(KernelFunc kernel,
                          dim3 grid, dim3 block,
                          const float* dA, const float* dB, float* dOut,
                          int M, int K,
                          int warmup, int iters) {
    for (int i = 0; i < warmup; ++i) kernel<<<grid, block>>>(dA, dB, dOut, M, K);
    CUDA_CHECK(cudaGetLastError());
    CUDA_CHECK(cudaDeviceSynchronize());

    cudaEvent_t start, stop;
    CUDA_CHECK(cudaEventCreate(&start));
    CUDA_CHECK(cudaEventCreate(&stop));
    CUDA_CHECK(cudaEventRecord(start));
    for (int i = 0; i < iters; ++i) kernel<<<grid, block>>>(dA, dB, dOut, M, K);
    CUDA_CHECK(cudaEventRecord(stop));
    CUDA_CHECK(cudaGetLastError());
    CUDA_CHECK(cudaEventSynchronize(stop));

    float ms = 0.0f;
    CUDA_CHECK(cudaEventElapsedTime(&ms, start, stop));
    CUDA_CHECK(cudaEventDestroy(start));
    CUDA_CHECK(cudaEventDestroy(stop));
    return ms / iters;
}

// overload for const-B kernel (no dB)
static float timeKernelMsConstB(dim3 grid, dim3 block,
                                const float* dA, float* dOut,
                                int M, int K, int warmup, int iters) {
    for (int i = 0; i < warmup; ++i) dotUnrollRegisterConstB<<<grid, block>>>(dA, dOut, M, K);
    CUDA_CHECK(cudaGetLastError());
    CUDA_CHECK(cudaDeviceSynchronize());

    cudaEvent_t start, stop;
    CUDA_CHECK(cudaEventCreate(&start));
    CUDA_CHECK(cudaEventCreate(&stop));
    CUDA_CHECK(cudaEventRecord(start));
    for (int i = 0; i < iters; ++i) dotUnrollRegisterConstB<<<grid, block>>>(dA, dOut, M, K);
    CUDA_CHECK(cudaEventRecord(stop));
    CUDA_CHECK(cudaGetLastError());
    CUDA_CHECK(cudaEventSynchronize(stop));

    float ms = 0.0f;
    CUDA_CHECK(cudaEventElapsedTime(&ms, start, stop));
    CUDA_CHECK(cudaEventDestroy(start));
    CUDA_CHECK(cudaEventDestroy(stop));
    return ms / iters;
}

static double estimateGFLOPS(int M, int K, float ms) {
    // one dot = K mul + (K-1) add ~ 2K ops (rough)
    double flops = 2.0 * (double)M * (double)K;
    return flops / (ms * 1e-3) / 1e9;
}

int main() {
    // Tune workload
    const int M = 1 << 20;  // number of rows
    const int K = 1024;     // dot length
    const float tol = 1e-5f;

    if (K > MAX_K) {
        fprintf(stderr, "Error: K=%d > MAX_K=%d. Increase MAX_K.\n", K, MAX_K);
        return EXIT_FAILURE;
    }

    const size_t bytesA = (size_t)M * K * sizeof(float);
    const size_t bytesB = (size_t)K * sizeof(float);
    const size_t bytesO = (size_t)M * sizeof(float);

    // Host
    float* hA   = (float*)std::malloc(bytesA);
    float* hB   = (float*)std::malloc(bytesB);
    float* hRef = (float*)std::malloc(bytesO);
    float* hOut = (float*)std::malloc(bytesO);
    if (!hA || !hB || !hRef || !hOut) {
        fprintf(stderr, "Host malloc failed.\n");
        return EXIT_FAILURE;
    }

    for (int i = 0; i < M * K; ++i) hA[i] = 0.001f * (i % 1000);
    for (int i = 0; i < K; ++i)     hB[i] = 0.002f * (i % 1000);

    // CPU ref
    dotCPU(hA, hB, hRef, M, K);

    // Device
    float *dA=nullptr, *dB=nullptr, *dOut=nullptr;
    CUDA_CHECK(cudaMalloc(&dA, bytesA));
    CUDA_CHECK(cudaMalloc(&dB, bytesB));
    CUDA_CHECK(cudaMalloc(&dOut, bytesO));
    CUDA_CHECK(cudaMemcpy(dA, hA, bytesA, cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(dB, hB, bytesB, cudaMemcpyHostToDevice));

    // Copy B to constant memory
    CUDA_CHECK(cudaMemcpyToSymbol(Bc, hB, bytesB));

    // Launch config
    int blockSize = 256;
    int gridSize  = (M + blockSize - 1) / blockSize;
    dim3 block(blockSize, 1, 1);
    dim3 grid(gridSize, 1, 1);

    const int warmup = 5;
    const int iters  = 20;

    // ---------------- Baseline ----------------
    dotBaseline<<<grid, block>>>(dA, dB, dOut, M, K);
    CUDA_CHECK(cudaGetLastError());
    CUDA_CHECK(cudaDeviceSynchronize());
    CUDA_CHECK(cudaMemcpy(hOut, dOut, bytesO, cudaMemcpyDeviceToHost));
    bool ok0 = checkClose(hOut, hRef, M, tol);

    float ms0 = timeKernelMs(dotBaseline, grid, block, dA, dB, dOut, M, K, warmup, iters);
    double g0 = estimateGFLOPS(M, K, ms0);

    // ---------------- Unroll+Reg (global B) ----------------
    dotUnrollRegisterGlobalB<<<grid, block>>>(dA, dB, dOut, M, K);
    CUDA_CHECK(cudaGetLastError());
    CUDA_CHECK(cudaDeviceSynchronize());
    CUDA_CHECK(cudaMemcpy(hOut, dOut, bytesO, cudaMemcpyDeviceToHost));
    bool ok1 = checkClose(hOut, hRef, M, tol);

    float ms1 = timeKernelMs(dotUnrollRegisterGlobalB, grid, block, dA, dB, dOut, M, K, warmup, iters);
    double g1 = estimateGFLOPS(M, K, ms1);

    // ---------------- Unroll+Reg (const B) ----------------
    dotUnrollRegisterConstB<<<grid, block>>>(dA, dOut, M, K);
    CUDA_CHECK(cudaGetLastError());
    CUDA_CHECK(cudaDeviceSynchronize());
    CUDA_CHECK(cudaMemcpy(hOut, dOut, bytesO, cudaMemcpyDeviceToHost));
    bool ok2 = checkClose(hOut, hRef, M, tol);

    float ms2 = timeKernelMsConstB(grid, block, dA, dOut, M, K, warmup, iters);
    double g2 = estimateGFLOPS(M, K, ms2);

    // Report
    printf("\n=== Dot product compare (M=%d, K=%d, UNROLL=%d) ===\n", M, K, UNROLL);
    printf("[1] Baseline                 : %s | %.4f ms | %.2f GFLOP/s\n", ok0 ? "PASS" : "FAIL", ms0, g0);
    printf("[2] Unroll+Reg (global B)    : %s | %.4f ms | %.2f GFLOP/s\n", ok1 ? "PASS" : "FAIL", ms1, g1);
    printf("[3] Unroll+Reg (const  Bc)   : %s | %.4f ms | %.2f GFLOP/s\n", ok2 ? "PASS" : "FAIL", ms2, g2);

    // Cleanup
    CUDA_CHECK(cudaFree(dA));
    CUDA_CHECK(cudaFree(dB));
    CUDA_CHECK(cudaFree(dOut));
    std::free(hA);
    std::free(hB);
    std::free(hRef);
    std::free(hOut);

    return (ok0 && ok1 && ok2) ? EXIT_SUCCESS : EXIT_FAILURE;
}


Writing unroll_register_compare.cu


In [27]:
!nvcc -arch=sm_80 unroll_register_compare.cu -o unroll_register_compare
!./unroll_register_compare


=== Dot product compare (M=1048576, K=1024, UNROLL=4) ===
[1] Baseline                 : PASS | 8.4628 ms | 253.76 GFLOP/s
[2] Unroll+Reg (global B)    : PASS | 7.3327 ms | 292.86 GFLOP/s
[3] Unroll+Reg (const  Bc)   : PASS | 7.3741 ms | 291.22 GFLOP/s
